<h1 style="color:rgb(255, 98, 0);">1. Why Pydantic</h1>


Pydantic is used in FastAPI to:

👉 Validate and parse input data automatically (e.g., JSON to Python objects)

👉 Ensure data types are correct using Python type hints

👉 Raise clear errors when data is invalid

In short:

Pydantic = Data validation + Conversion + Error handling ✅

Here, in below example we have to write too much of manual code when we scale up. Which is not suitable for high scale projects hence Pydantic came into picture.

In [5]:
def insert_patient_data(name : str, age: int):
    if type(name) == str and type(age) == int:
        if age < 0:
            raise ValueError("Age cannot be negative.")
        else:
            return {"name": name, "age": age}
    else:
        raise TypeError("Invalid data types: name must be a string and age must be an integer.")

# if not correctly typed, it will raise a TypeError
insert_patient_data("John Doe", 30)

{'name': 'John Doe', 'age': 30}

In [6]:
def update_patient_data(name: str, age: int):
    if type(name) == str and type(age) == int:
        if age < 0:
            raise ValueError("Age cannot be negative.")
        else:
            return {"name": name, "age": age}
    else:
        raise TypeError("Invalid data types: name must be a string and age must be an integer.")

### Type Validation

In [7]:
from pydantic import BaseModel
from typing import List, Dict, Optional

class Patient(BaseModel):
    name: str
    age: int
    weight: float
    married: bool
    # we used typing module in order to check the type of data stored
    allergies: Optional[List[str]] = None # We have to use None as default value for optional fields
    contact_info: Dict[str, str]

def insert_patient_data(patient: Patient):
    print(patient.name)
    print(patient.age)
    print('Inserted')

def update_patient_data(patient: Patient):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print('Updated')

patient_info = {'name': "John Doe" , 'age': '30', 'weight': 75.2, 'married': True, 'contact_info':{'email':'abc@gmail.com', 'phone':'123456789'}} # This 30 will get converted to int by Pydantic

patient1 = Patient(**patient_info)

update_patient_data(patient1)

John Doe
30
None
Updated


### Data Validation

In [8]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):
    name: Annotated[str, Field(max_length=50, title='Name of the patient', description='Give the name of the patient in less than 50 chars', examples=['Nitish', 'Amit'])]
    email: EmailStr
    linkedin_url: AnyUrl
    age: int
    weight: Annotated[float, Field(gt=0, strict=True)] # strict=True ensures that the value is a float and not a string
    married: Annotated[bool, Field(default=None, description='Is the patient married or not')]
    # we used typing module in order to check the type of data stored
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)] # We have to use None as default value for optional fields
    contact_info: Dict[str, str]

def update_patient_data(patient: Patient):
    print(patient.name)
    print(patient.age)
    print(patient.email)
    print(patient.linkedin_url)
    print('Updated')

patient_info = {'name': "John Doe" , 'age': '30', 'email':'abc@gmail.com', 'linkedin_url':'https://www.youtube.com/watch?v=lRArylZCeOs&t=119s' ,'weight': 75.2, 'married': True, 'contact_info':{'phone':'123456789'}} # This 30 will get converted to int by Pydantic

patient1 = Patient(**patient_info)

update_patient_data(patient1)

John Doe
30
abc@gmail.com
https://www.youtube.com/watch?v=lRArylZCeOs&t=119s
Updated


<h1 style="color:rgb(255, 98, 0);">2. Field Validator</h1>

Field Validator is used to:

👉 Add custom validation logic for a specific field or we can do custom transformations as well

👉 Ensure values meet rules that can’t be handled by standard Field() constraints

In short:

Field Validator = Custom rule checker for one field ✅

In [11]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field, field_validator
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator('email')
    @classmethod
    def validate_email(cls, value):
        valid_email_domains = ['hdfc.com', 'icici.com']
        
        # abc@gmail.com
        domain_name = value.split('@')[-1]

        if domain_name not in valid_email_domains:
            raise ValueError(f"Email domain must be one of {valid_email_domains}.")
        
    @field_validator('name')
    @classmethod
    def validate_name(cls, value):
        return value.upper()

def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {'name':'Aadish', 'email':'abc@icici.com', 'age': '30', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462'}}

patient1 = Patient(**patient_info) # validation -> type coercion

update_patient_data(patient1)

AADISH
30
['pollen', 'dust']
True
updated


<h1 style="color:rgb(255, 98, 0);">3. Model Validator</h1>

Model Validator is used to:

👉 Apply validation across multiple fields

👉 Check relationships between fields (e.g., end_date > start_date)

👉 Perform post-init logic after the full model is created

In short:

Model Validator = Validate multiple fields together ✅

In [12]:
from pydantic import BaseModel, EmailStr, model_validator
from typing import List, Dict

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(cls, model):
        if model.age > 60 and 'emergency' not in model.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact')
        return model

def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {'name':'nitish', 'email':'abc@icici.com', 'age': '65', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

patient1 = Patient(**patient_info) 

update_patient_data(patient1)

nitish
65
['pollen', 'dust']
True
updated


<h1 style="color:rgb(255, 98, 0);">4. Computed Field</h1>

In [13]:
from pydantic import BaseModel, EmailStr, computed_field
from typing import List, Dict

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float # kg
    height: float # mtr
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @computed_field
    @property
    def bmi(self) -> float:
        bmi = round(self.weight/(self.height**2),2)
        return bmi

def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('BMI', patient.bmi)
    print('updated')

patient_info = {'name':'nitish', 'email':'abc@icici.com', 'age': '65', 'weight': 75.2, 'height': 1.72, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

patient1 = Patient(**patient_info) 

update_patient_data(patient1)

nitish
65
['pollen', 'dust']
True
BMI 25.42
updated


<h1 style="color:rgb(255, 98, 0);">5. Nested Models</h1>

In [15]:
from pydantic import BaseModel

class Address(BaseModel):

    city: str
    state: str
    pin: str

class Patient(BaseModel):

    name: str
    gender: str
    age: int
    address: Address

address_dict = {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}

address1 = Address(**address_dict)

patient_dict = {'name': 'nitish', 'gender': 'male', 'age': 35, 'address': address1}

patient1 = Patient(**patient_dict)

temp = patient1.model_dump()

print(temp)

# Better organization of related data (e.g., vitals, address, insurance)

# Reusability: Use Vitals in multiple models (e.g., Patient, MedicalRecord)

# Readability: Easier for developers and API consumers to understand

# Validation: Nested models are validated automatically—no extra work needed

{'name': 'nitish', 'gender': 'male', 'age': 35, 'address': {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}}
